In [112]:
import re
import sys
import nltk
import string
from math import sqrt, log
from collections import defaultdict
from itertools import chain, product
from nltk import word_tokenize as tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer as stemmer
import numpy as np
from itertools import groupby
from collections import Counter
from nltk.collocations import *
from nltk.stem.porter import PorterStemmer
import pandas as pd
import textfeatures as tf
from sklearn import preprocessing
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from textstat.textstat import textstatistics, easy_word_set, legacy_round

ModuleNotFoundError: No module named 'textstat'

In [121]:
def compute_average_word_length(sentence):
    return np.mean([len(words) for words in sentence.split()])

def compute_average_sentence_length(sentence):
    sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    return np.mean([len(words) for words in sentence])

def freq_of_words_great_sent_len(sentence):
    result = []
    avg_word_len = compute_average_word_length(sentence)
    # sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    sentence = Counter(sentence.split())
    for key, value in sentence.items():
        if len(key) > avg_word_len:
            result.append(value)
#             print (key, value)
    return sum(result)

def tokenize(sentence):
    return re.split(r"[^0-9A-Za-z\-'_]+", sentence)

def compute_yules_k_for_text(sentence):
    tokens = tokenize(sentence)
    counter = Counter(token.upper() for token in tokens)

    #compute number of word forms in a given sentence/text
    m1 = sum(counter.values())
    m2 = sum([frequency ** 2 for frequency in counter.values()])

    #compute yules k measure and return the value
    yules_k = 10000/((m1 * m1) / (m2 - m1))
    return yules_k


def words_in_sentence(sentence):
    w = [words.strip("0123456789!:,.?()[]{}") for words in sentence.split()]
    return filter(lambda x: len(x) > 0, w)

def compute_yules_i_for_text(sentence):
    dictionary = {}
    stemmer = PorterStemmer()

    for word in words_in_sentence(sentence):
        word = stemmer.stem(word).lower()
        try:
            dictionary[word] += 1
        except:
            dictionary[word] = 1

    m1 = float(len(dictionary))
    m2 = sum([len(list(grouped_values)) * (frequency ** 2) for frequency, grouped_values in groupby(sorted(dictionary.values()))])

    # compute yules i and return the value
    try:
        yules_i = (m1 * m1) / (m2 - m1)
        return yules_i
    except ZeroDivisionError:
        return 0
    
# polysemy for the content words
def polysemy(group):
    stop = stopwords.words('english')
    str1 = [i for i in group.split() if i not in stop]
    a = list()
    for w in str1:
        if(len(wn.synsets(w)) > 1):
            a.append(w)
    return len(a)

# Returns average sentence length
def avg_sentence_length(self, text):
    words = word_count(text)
    sentences = sentence_count(text)
    average_sentence_length = float(words / sentences)
    return average_sentence_length
def syllables_count(word):
    return textstatistics().syllable_count(word)
# Returns the average number of syllables per
# word in the text
def avg_syllables_per_word(text):
    syllable = syllables_count(text)
    words = word_count(text)
    ASPW = float(syllable) / float(words)
def break_sentences(text):
    nlp = spacy.load('en')
    doc = nlp(text)
    return doc.sents
# Returns Number of Words in the text
def word_count(text):
    sentences = break_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words
# Returns the number of sentences in the text
def sentence_count(text):
    sentences = break_sentences(text)

def flesch_reading_ease(text):
    """
        Implements Flesch Formula:
        Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
        Here,
          ASL = average sentence length (number of words
                divided by number of sentences)
          ASW = average word length in syllables (number of syllables
                divided by number of words)
    """
    FRE = 206.835 - float(1.015 * avg_sentence_length(text)) -\
          float(84.6 * avg_syllables_per_word(text))
    return legacy_round(FRE, 2)

In [122]:
raw_data = pd.read_csv("scores.csv")
# raw_data = raw_data.sample(frac=0.01).reset_index(drop=True)
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return content

In [123]:
raw_data["yules_i_measure_abs"] = np.nan
raw_data["yules_i_measure_abs"] = raw_data.apply(lambda x: compute_yules_i_for_text(x['abstract']), axis=1)

raw_data["avg_word_len_abs"] = np.nan
raw_data["avg_word_len_abs"] = raw_data.apply(lambda x: compute_average_word_length(x['abstract']), axis=1)

raw_data["avg_sen_len_abs"] = np.nan
raw_data["avg_sen_len_abs"] = raw_data.apply(lambda x: compute_average_sentence_length(x['abstract']), axis=1)

raw_data["freq_words_gt_sen_len_abs"] = np.nan
raw_data["freq_words_gt_sen_len_abs"] = raw_data.apply(lambda x: freq_of_words_great_sent_len(x['abstract']), axis=1)

raw_data["polysemy"] = np.nan
raw_data["polysemy"] = raw_data.apply(lambda x: freq_of_words_great_sent_len(x['abstract']), axis=1)

raw_data["flesch_reading_ease"] = np.nan
raw_data["flesch_reading_ease"] = raw_data.apply(lambda x: flesch_reading_ease(x['abstract']), axis=1)

raw_data.head()

TypeError: avg_sentence_length() missing 1 required positional argument: 'text'

In [ ]:
data = raw_data.copy()
data['similarity_score'] = pd.to_numeric(data['semantic_similarity_aftercleaning'].str.replace('[','').str.replace(']',''))
data = data[data['similarity_score'] != 999999999.0].copy()
data['similarity_score'] = data['similarity_score'] * 100

In [ ]:
tf.word_count(data,"abstract","word_cnt")
tf.char_count(data,"abstract","char_len")
tf.avg_word_length(data,"abstract","avg_wrd_length")
tf.stopwords_count(data,"abstract","stopwords_cnt")
tf.numerics_count(data,"abstract","num_len")

In [124]:
columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs', 'similarity_score',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len', 'polysemy', 'flesch_reading_ease']
df = data[columns].copy()
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
#df = pd.DataFrame(x_scaled)
df=pd.DataFrame(x_scaled, columns=df.columns)

KeyError: "['flesch_reading_ease'] not in index"

In [97]:
training_set = data.sample(frac=0.8, random_state = 1)
test_set = data.loc[~data.index.isin(training_set.index)]
test_set.head()

training_set = df.sample(frac=0.8, random_state = 1)
test_set = df.loc[~df.index.isin(training_set.index)]
test_set.head()

,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len,polysemy
14,0.283638,0.359858,0.383752,0.791946,0.860262,0.661721,0.695251,0.359858,0.492537,0.0,0.791946
15,0.353237,0.489666,0.305471,0.302013,0.650185,0.255193,0.284002,0.489666,0.194030,0.0,0.302013
18,0.168668,0.601191,0.473236,0.389262,0.774737,0.403561,0.467036,0.601191,0.358209,0.0,0.389262
20,0.372270,0.771313,0.273380,0.268456,0.723092,0.201780,0.264638,0.771313,0.164179,0.0,0.268456
25,0.352939,0.684665,0.314639,0.261745,0.793084,0.231454,0.289534,0.684665,0.171642,0.0,0.261745


In [98]:
data_columns = ['yules_i_measure_abs', 'avg_word_len_abs',
       'avg_sen_len_abs', 'freq_words_gt_sen_len_abs', 'polysemy']

columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len', 'polysemy']


# training_data = training_set.as_matrix(columns = data_columns)
training_data = training_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(training_data)
training_data[nan_locs] = 0

training_target = training_set['similarity_score'].values
nan_locs = np.isnan(training_target)
training_target[nan_locs] = 0

test_data = test_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(test_data)
test_data[nan_locs] = 0

test_target = test_set['similarity_score'].values
nan_locs = np.isnan(test_target)
test_target[nan_locs] = 0

In [99]:
train_data_array = training_data.copy()
train_class_array = training_target.copy()
test_data_array = test_data.copy()
test_class_array = test_target.copy()

In [100]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor()
reg.fit(training_data, training_target)

print('R squared: ' + str(reg.score(test_data, test_target)))
pred = reg.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.12353661053259768
MAE: 0.09264346028057945
MSE: 0.013884759816458142


In [101]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(training_data, training_target)

print('R squared: ' + str(svr.score(test_data, test_target)))
pred = svr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.14649566647001444
MAE: 0.09078776941844129
MSE: 0.013521046989277339


In [102]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge()
kr.fit(training_data, training_target)

print('R squared: ' + str(kr.score(test_data, test_target)))
pred = kr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.8472842498037556
MAE: 0.13597469011441832
MSE: 0.029264312040275083


In [103]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(training_data, training_target)

print('R squared: ' + str(dt.score(test_data, test_target)))
pred = dt.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.16370423905680132
MAE: 0.10309986957717793
MSE: 0.018435172593479807


In [104]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 2000)
rf.fit(training_data, training_target)

print('R squared: ' + str(rf.score(test_data, test_target)))
pred = rf.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.05687087074336494
MAE: 0.09482643969027843
MSE: 0.014940865292264124


In [105]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(training_data, training_target)

print('R squared: ' + str(gb.score(test_data, test_target)))
pred = gb.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.12965718103564206
MAE: 0.09017677372147236
MSE: 0.013787798948045703


In [106]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pr = Pipeline([('poly', PolynomialFeatures(degree = 3)),
              ('linear', LinearRegression(fit_intercept = False))])

pr.fit(training_data, training_target)

print('R squared: ' + str(pr.score(test_data, test_target)))
pred = pr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.1342978368299239
MAE: 0.0915514181378897
MSE: 0.013714282596000911


In [107]:
from sklearn.linear_model import Lasso
lm = Lasso()
lm.fit(training_data, training_target)

print('R squared: ' + str(lm.score(test_data, test_target)))
pred = lm.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.0005374783233043079
MAE: 0.09928753141715577
MSE: 0.01585031701361265


In [108]:
from sklearn.linear_model import ElasticNet
en = ElasticNet()
en.fit(training_data, training_target)

print('R squared: ' + str(en.score(test_data, test_target)))
pred = en.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.0005374783233043079
MAE: 0.09928753141715577
MSE: 0.01585031701361265


In [109]:
from sklearn.linear_model import LassoLars
ll = LassoLars()
ll.fit(training_data, training_target)

print('R squared: ' + str(ll.score(test_data, test_target)))
pred = ll.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.0005374783233043079
MAE: 0.09928753141715577
MSE: 0.01585031701361265


In [110]:
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(training_data, training_target)

print('R squared: ' + str(br.score(test_data, test_target)))
pred = br.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.09164383039022828
MAE: 0.0936266240301292
MSE: 0.014389998937084725


In [111]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(training_data, training_target)

print('R squared: ' + str(sgd.score(test_data, test_target)))
pred = sgd.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.011293623040671719
MAE: 0.09887368733240863
MSE: 0.01566289104376946


In [ ]:
\begin{table}[ht]
\caption{Model Results} % title of Table
\centering % used for centering table
\begin{tabular}{c c c c} % centered columns (4 columns)
\hline\hline %inserts double horizontal lines
Case & Method\#1 & Method\#2 & Method\#3 \\ [0.5ex] % inserts table
%heading
\hline % inserts single horizontal line
1 & 50 & 837 & 970 \\ % inserting body of the table
2 & 47 & 877 & 230 \\
3 & 31 & 25 & 415 \\
4 & 35 & 144 & 2356 \\
5 & 45 & 300 & 556 \\ [1ex] % [1ex] adds vertical space
\hline %inserts single line
\end{tabular}
\label{table:nonlin} % is used to refer this table in the text
\end{table}
--------------------- e

In [64]:
from sklearn.model_selection import RandomizedSearchCV
# Setup the parameters and distributions to sample from: param_dist
param_dist_dt = {"max_depth": [1, 32],
              "max_features": list(range(1, 21)),
              "random_state": list(range(1, 5000)),
              "min_samples_split": np.linspace(0.1, 1, 10, endpoint=True),
              "min_samples_leaf": np.linspace(0.1, 0.5, 5, endpoint=True),
              "criterion": ["friedman_mse", "mse"]}
## Instantiate the RandomizedSearchCV object: tree_cv
dt_cv = RandomizedSearchCV(DecisionTreeRegressor(), \
                           param_dist_dt, cv=10, n_jobs=-1, verbose=1)
# Fit it to the data
dt_cv.fit(train_data_array, train_class_array)
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(dt_cv.best_params_))
print("Best score is {}".format(dt_cv.best_score_))
# train mse
pred = dt_cv.predict(train_data_array)
score = metrics.mean_squared_error(train_class_array, pred)
print("Train MSE:   %0.3f" % score)
# test mse
pred = dt_cv.predict(test_data_array)
score = metrics.mean_squared_error(test_class_array, pred)
print("Test MSE:   %0.3f" % score)
# test mae
pred = dt_cv.predict(test_data_array)
score = metrics.mean_absolute_error(test_class_array, pred)
print("Test MAE:   %0.3f" % score)
# r-squared
score = metrics.r2_score(test_class_array, pred)
print("R-squared  %0.3f" % score)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Tuned Decision Tree Parameters: {'random_state': 1558, 'min_samples_split': 0.9, 'min_samples_leaf': 0.30000000000000004, 'max_features': 2, 'max_depth': 32, 'criterion': 'mse'}
Best score is 0.06330274150685954
Train MSE:   0.014
Test MSE:   0.014
Test MAE:   0.093
R-squared  0.040


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [88]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"learning_rate": [1, 0.5, 0.25, 0.1, 0.05, 0.01],
              "n_estimators": [1, 2, 4, 8, 16, 32, 64, 100, 200],
              "random_state": list(range(1, 5000)),
              "max_features": list(range(1, 9)),
              "max_depth": np.linspace(1, 32, 32, endpoint=True),
              "min_samples_split": np.linspace(0.1, 1.0, 10, endpoint=True),
              "min_samples_leaf": np.linspace(0.1, 0.5, 5, endpoint=True)}
# Instantiate a Decision Tree classifier: tree
gdb = GradientBoostingRegressor()
# Instantiate the RandomizedSearchCV object: tree_cv
gdb_cv = RandomizedSearchCV(gdb, param_dist, cv=10, n_jobs=-1, verbose=1)
# Fit it to the data
gdb_cv.fit(train_data_array, train_class_array)
# Print the tuned parameters and score
print("Tuned Gradient Boosting Parameters: {}".format(gdb_cv.best_params_))
print("Best score is {}".format(rf_cv.best_score_))
# train mse
pred = gdb_cv.predict(train_data_array)
score = metrics.mean_squared_error(train_class_array, pred)
print("Train MSE:   %0.3f" % score)
# test mse
pred = gdb_cv.predict(test_data_array)
score = metrics.mean_squared_error(test_class_array, pred)
print("Test MSE:   %0.3f" % score)
# test mae
pred = gdb_cv.predict(test_data_array)
score = metrics.mean_absolute_error(test_class_array, pred)
print("Test MAE:   %0.3f" % score)
# r-squared
score = metrics.r2_score(test_class_array, pred)
print("R-squared  %0.3f" % score)

OverflowError: Python int too large to convert to C long